In [73]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract article title
    article_title = soup.title.text.strip()

    # Extract article text
    article_text = ''
    for paragraph in soup.find_all('p'):
        article_text += paragraph.text + '\n'

    return article_title, article_text

def main():
    # Read URLs from input.xlsx
    input_file = 'Input.xlsx'
    df = pd.read_excel(input_file)

    for index, row in df.iterrows():
        url_id = row['URL_ID']
        url = row['URL']

        # Extract article text
        article_title, article_text = extract_article_text(url)

        # Save the extracted article to a text file
        output_file = f'{url_id}.txt'
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(f'{article_title}\n\n{article_text}')

if __name__ == "__main__":
    main()


ConnectionError: HTTPSConnectionPool(host='insights.blackcoffer.com', port=443): Max retries exceeded with url: /rising-it-cities-and-its-impact-on-the-economy-environment-infrastructure-and-city-life-by-the-year-2040-2/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000020B80D21F70>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [75]:
# import pandas as pd
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from openpyxl import load_workbook
from nltk.corpus import cmudict

nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


def main():
    # Read URLs from input.xlsx
    input_file = 'Input.xlsx'
    df_input = pd.read_excel(input_file)

    # Create a list to store the results
    results = []

    for index, row in df_input.iterrows():
        url_id = row['URL_ID']
        file_path = f'{url_id}.txt'

        # Read the text file
        with open(file_path, 'r', encoding='utf-8') as file:
            article_text = file.read()
        
        
    #positive & negative words
        
        with open('positive_words.txt') as file:
            pos = file.read().split()
            
        with open('negative_words.txt') as file:
            neg = file.read().split()
        
        positive_score = 0
        negative_score = 0
    
        for i in pos:
            for j in article_text.split():
                if (i==j):
                    positive_score = positive_score + 1
        
        for i in neg:
            for j in article_text.split():
                if (i==j):
                    negative_score = negative_score + 1
        
        
    #polarity score
        
        polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)    
        
        
    #subjectivity score
        
        total_words =  len(article_text.split()) - (positive_score + negative_score)
        subjectivity_score = (positive_score + negative_score) / (total_words + 0.000001)
        
        
    #average sentence length
        
        avg_sent_len = round(len(article_text) / (len(article_text.split('.'))))
        
        
    #complex words
        
        pronouncing_dict = cmudict.dict()
        def count_syllables(word):
        # Use the CMU Pronouncing Dictionary to count syllables
            try:
                return [len(list(y for y in x if y[-1].isdigit())) for x in pronouncing_dict[word.lower()]][0]
            except KeyError:
                return 0

        def is_complex_word(word, threshold=2):
        # Adjust the threshold as needed
            return count_syllables(word) >= threshold
            
        comp_word = 0
        for i in article_text.split():
            if is_complex_word(i):
                comp_word += 1
                
        perc_comp_word = comp_word / len(article_text.split())
        
        
    #fog index
        
        fog_index = 0.4 * (avg_sent_len + perc_comp_word)
        
        
    #complex word count
        
        comp_word_count = round(perc_comp_word * 100)
        
        
    #total word count
        
        word_count = len(article_text.split())
        

        # Append results to the list
        results.append({
            'URL_ID': url_id,
            'Positive Score': positive_score,
            'Negative Score': negative_score,
            'Polarity Score': polarity_score,
            'Subjectivity Score': subjectivity_score,
            'Average Sentence Length': avg_sent_len,
            'Percentage of Complex words': perc_comp_word,
            'Fog Index': fog_index,
            'Complex Word Count': comp_word_count,
            'Word Count': word_count
        })

    # Create a DataFrame from the results list
    df_results = pd.DataFrame(results)

    # Save the results in the specified order
    output_file = 'Output_Data_Structure.xlsx'
    df_results.to_excel(output_file, index=False)

if __name__ == "__main__":
    main()